## VNS na Nearest String problemu

In [1]:
import random

In [2]:
%run utils.ipynb

### 1) Inicijalizacija

Početni string se generiše tako da na svakoj poziciji bira karakter iz skupa karaktera koji se pojavljuju u toj **„koloni“** ulaznih stringova.  
Time se startuje od smislenog kandidata koji je u skladu sa datim instancama.

In [3]:
def _initialize(words):
    alphabets = get_all_alphabets(words)
    return generate_random_string_from_alphabets(alphabets)

### 2) Shaking (diverzifikacija)

Za dato `k`, algoritam nasumično izabere `k` pozicija i promeni njihove karaktere.  
To predstavlja pretragu u **k-susedstvu**: rešenja koja se razlikuju u `k` karaktera.

- malo `k` → male, pažljive promene  
- veće `k` → veći „skokovi“ koji pomažu izlazak iz lokalnog minimuma  

In [4]:
def _change_char_random(word, alphabet, pos):
    new_char = random.choice(alphabet)
    return word[:pos] + new_char + word[pos + 1 :]

In [5]:
def _shaking(target, words, k):
    target_new = target
    idx = random.sample(range(len(target)), k)
    for i in idx:
        alphabet = get_column_alphabet(words, i)
        target_new = _change_char_random(target_new, alphabet, i)
    return target_new

### 3) Lokalna pretraga (intenzifikacija)

Nakon *shaking*-a, radi se lokalna pretraga koja pokušava da poboljša rešenje menjajući karaktere jedan po jedan i prihvata **prvo poboljšanje** (*first improvement*).  
Ovo brzo „izravnava“ rešenje i vodi ka lokalno najboljem kandidatu u datom regionu.

In [6]:
def _local_search_first_improvement(target, distance, words):
    improved = True
    while improved:
        improved = False
        for i in range(len(target)):
            alphabet = get_column_alphabet(words, i)
            target_new = _change_char_random(target, alphabet, i)
            distance_new = max_hamming_distance(target_new, words)
            if distance_new < distance:
                distance = distance_new
                target = target_new
                improved = True
                break
    return target, distance

### 4) Pravilo prihvatanja

Ako novo rešenje smanji vrednost funkcije cilja `f(s)`, ono se prihvata odmah.  
Ponekad se prihvata i jednako dobro rešenje (sa određenom verovatnoćom) kako bi se održala raznovrsnost i izbeglo stagniranje algoritma.

In [7]:
def _acceptance_rule(distance_new, distance):
    move_prob = 0.5
    return distance_new < distance or (
        distance_new == distance and random.random() < move_prob
    )

### 5) Promena susedstva

Ako nema napretka u manjem susedstvu, povećava se `k` (prelazak na **šire susedstvo**).  
Ako se pronađe bolje rešenje, algoritam se vraća na manje `k` i nastavlja intenzivno poboljšavanje.

In [8]:
def vns(words):
    m = len(words[0])
    k_min = 1
    k_max = min(3, m)
    if m > 3:
        k_max = random.randint(3, m)
    max_iter = 1000
    target = _initialize(words)
    distance = max_hamming_distance(target, words)
    it = 0
    while not it >= max_iter:
        it += 1
        for k in range(k_min, k_max):
            target_new = _shaking(target, words, k)
            distance_new = max_hamming_distance(target_new, words)
            target_new, distance_new = _local_search_first_improvement(
                target_new, distance_new, words
            )
            if _acceptance_rule(distance_new, distance):
                distance = distance_new
                target = target_new
                break

    return distance, target